In [2]:
import json
from itertools import islice
import pandas as pd
import torch
import torch.nn.functional as F
from transformers import (
    PreTrainedTokenizerFast,
    AutoModelForCausalLM,
    AutoTokenizer,
    )

# Upload tools

In [3]:
# deepseek_model_name = "deepseek-ai/DeepSeek-V2-Lite"
# deepseek_model = AutoModelForCausalLM.from_pretrained(deepseek_model_name,
#                                                       trust_remote_code=True)  


# deepseek_tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")
# deepseek_model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")


# deepseek_r1_tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1")
# deepseek_r1_model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1", trust_remote_code=True)  


# athene_tokenizer = AutoTokenizer.from_pretrained("Nexusflow/Athene-V2-Chat")
# athene_model = AutoModelForCausalLM.from_pretrained("Nexusflow/Athene-V2-Chat")

tokenizer_config.json:   0%|          | 0.00/3.06k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [4]:
# qwen_tokenizer_path = "/kaggle/input/tokenizers/qwen_tokenizer.json"
# deepseek_tokenizer_path = "/kaggle/input/tokenizers/deepseek_tokenizer.json"
# llama_tokenizer_path = "/kaggle/input/tokenizers/llama_tokenizer.json"

# with open(qwen_tokenizer_path, 'r') as file:
#     qwen_tokenizer = PreTrainedTokenizerFast(tokenizer_file=qwen_tokenizer_path)
#     qwen = json.load(file)
#     qwen_vocab = qwen['model']['vocab']
# with open(deepseek_tokenizer_path, 'r') as file:
#     deepseek_tokenizer = PreTrainedTokenizerFast(tokenizer_file=deepseek_tokenizer_path)
#     deepseek = json.load(file)
#     deepseek_vocab = deepseek['model']['vocab']
# with open(llama_tokenizer_path, 'r') as file:
#     llama_tokenizer = PreTrainedTokenizerFast(tokenizer_file=llama_tokenizer_path)
#     llama = json.load(file)
#     llama_vocab = llama['model']['vocab']

In [3]:
python_keywords = [
    "def", "class", "import", "from", "as", "if", "elif", "else", "while", 
    "for", "in", "break", "continue", "return", "yield", "try", "except", 
    "finally", "with", "assert", "lambda", "global", "nonlocal", "pass", 
    "raise", "True", "False", "None", "is", "not"
]

java_keywords = [
    "class", "interface", "extends", "implements", "package", "import", "public", 
    "private", "protected", "static", "final", "abstract", "synchronized", 
    "volatile", "transient", "native", "strictfp", "void", "int", "double", 
    "float", "char", "boolean", "long", "short", "byte", "if", "else", "while",
    "for"
]


cpp_keywords = [
    "int", "float", "double", "char", "void", "bool", "short", "long", "signed", 
    "unsigned", "if", "else", "switch", "case", "default", "for", "while", 
    "do", "break", "continue", "return", "goto", "class", "struct", "union", 
    "namespace", "using", "private", "protected", "public"
]

go_keywords = [
    "break", "case", "chan", "const", "continue", "default", "defer", "else", 
    "fallthrough", "for", "func", "go", "goto", "if", "import", "interface", 
    "map", "package", "range", "return", "select", "struct", "switch", "type", 
    "var", "bool", "byte", "complex64", "complex128", "float32"
]

react_keywords = [
    "React", "Component", "useState", "useEffect", "useContext", "useReducer", 
    "useMemo", "useCallback", "useRef", "useLayoutEffect", "useImperativeHandle", 
    "jsx", "props", "state", "context", "provider", "consumer", "hook", 
    "fragment", "key", "ref", "render", "virtualDOM", "reconciliation", 
    "portal", "suspense", "lazy", "errorBoundary", "children", "strictMode"
]

javascript_keywords = [
    "var", "let", "const", "if", "else", "switch", "case", "default", "for", 
    "while", "do", "break", "continue", "return", "function", "class", 
    "extends", "constructor", "super", "import", "export", "try",
    "catch", "finally", "throw", "this", "new", "delete", "instanceof", "typeof"
]

rust_keywords = [
    "as", "break", "const", "continue", "crate", "else", "enum", "extern", 
    "false", "fn", "for", "if", "impl", "in", "let", "loop", "match", 
    "mod", "move", "mut", "pub", "ref", "return", "Self", "self", "static", 
    "struct", "super", "trait", "true"
]

php_keywords = [
    "abstract", "and", "array", "as", "break", "callable", "case", "catch", 
    "class", "clone", "const", "continue", "declare", "default", "do", "echo", 
    "else", "elseif", "empty", "enddeclare", "endfor", "endforeach", "endif", 
    "endswitch", "endwhile", "eval", "exit", "extends", "final", "finally"
]

all_keywords = [python_keywords,
                java_keywords,
                cpp_keywords,
                go_keywords,
                react_keywords,
                javascript_keywords,
                rust_keywords,
                php_keywords,]

In [4]:
misswords = ['nonlocal',
             'synchronized',
             'transient',
             'strictfp',
             'fallthrough',
             'complex64',
             'complex128',
             'float32',
             'useEffect',
             'useContext',
             'useReducer',
             'useMemo',
             'useCallback',
             'useRef',
             'useLayoutEffect',
             'useImperativeHandle',
             'virtualDOM',
             'reconciliation',
             'suspense',
             'errorBoundary',
             'useStrictMode',
             'instanceof',
             'enddeclare',
             'endfor',
             'endswitch',
             'endwhile',
            ]

# Warm start

In [6]:
def get_token_probabilities(model, tokenizer, text, top_k=None):
    """
    Get token probabilities from a GPT model for each position in the input text.

    Args:
        model: model instance
        tokenizer: tokenizer instance
        text: Input text to analyze
        top_k: Optional, number of top probabilities to return for each position

    Returns:
        List of dictionaries containing token probabilities for each position
    """
    # Tokenize input text
    input_ids = tokenizer.encode(text, return_tensors='pt')

    # Get model's raw predictions
    with torch.no_grad():
        outputs = model(input_ids)
        logits = outputs.logits

    # Convert logits to probabilities using softmax
    probs = F.softmax(logits[0], dim=-1)

    # Store results for each position
    token_probs = []

    # Analyze each position
    for position in range(len(input_ids[0])):
        position_probs = probs[position]

        # Get token indices sorted by probability
        sorted_indices = torch.argsort(position_probs, descending=True)

        # Limit to top_k if specified
        if top_k:
            sorted_indices = sorted_indices[:top_k]

        # Create dictionary of token:probability pairs
        position_dict = {
            'position': position,
            'token': tokenizer.decode(input_ids[0][position]),
            'probabilities': {
                tokenizer.decode([idx.item()]): position_probs[idx].item()
                for idx in sorted_indices
            }
        }

        token_probs.append(position_dict)

    return token_probs

In [14]:
def get_token_probabilities_gpu(model, tokenizer, text, top_k=None):
    """
    Get token probabilities from a GPT model for each position in the input text.

    Args:
        model: GPT model instance
        tokenizer: GPT tokenizer instance
        text: Input text to analyze
        top_k: Optional, number of top probabilities to return for each position

    Returns:
        List of dictionaries containing token probabilities for each position
    """
    input_ids = tokenizer.encode(text, return_tensors='pt').to(device)

    with torch.no_grad():
        outputs = model(input_ids)
        logits = outputs.logits

    probs = F.softmax(logits[0], dim=-1)
    token_probs = []

    for position in range(len(input_ids[0])):
        position_probs = probs[position]

        sorted_indices = torch.argsort(position_probs, descending=True)
        
        if top_k:
            sorted_indices = sorted_indices[:top_k]

        position_dict = {
            'position': position,
            'token': tokenizer.decode(input_ids[0][position]),
            'probabilities': {
                tokenizer.decode([idx.item()]): position_probs[idx].item()
                for idx in sorted_indices
            }
        }

        token_probs.append(position_dict)

    return token_probs

In [15]:
# Example usage
def analyze_text_gpu(text, model, tokenizer, top_k=50):
    """
    Analyze token probabilities for a given text using specified GPT model.

    Args:
        text: Input text to analyze
        model_name: Name of the GPT model to use
        top_k: Number of top probabilities to show for each position
    """
    # Load model and tokenizer
    # model = GPT2LMHeadModel.from_pretrained(model_name)
    # tokenizer = GPT2Tokenizer.from_pretrained(model_name)

    # Get token probabilities
    results = get_token_probabilities_gpu(model, tokenizer, text, top_k)
    
    # Print results
    # print(f"Analysis of text: '{text}'\n")
    # for pos_data in results[:5]:
    #     print(f"Position {pos_data['position']}: Token '{pos_data['token']}'")
    #     print("Top probabilities:")
    #     for token, prob in islice(pos_data['probabilities'].items(), 3):
    #         print(f"->{token}<-: {prob:.4f}")
    #     print()
        
    tokens_prob = {}
    for i in range(len(results)):
        given_token = results[i]['token']
        tokens_prob[given_token] = {}
        
        for token, prob in islice(results[i]['probabilities'].items(), 3):
            tokens_prob[given_token][token] = prob

    return tokens_prob

In [7]:
# Example usage
def analyze_text(text, model, tokenizer, top_k=50):
    """
    Analyze token probabilities for a given text using specified model.

    Args:
        text: Input text to analyze
        model: Model to use
        tokenizer: Tokenizer to use
        top_k: Number of top probabilities to show for each position
    """

    # Get token probabilities
    results = get_token_probabilities(model, tokenizer, text, top_k)

    tokens_prob = {}
    for i in range(len(results)):
        given_token = results[i]['token']
        tokens_prob[given_token] = {}
        
        for token, prob in islice(results[i]['probabilities'].items(), 10):
            tokens_prob[given_token][token] = prob

    return tokens_prob

In [22]:
algorithms = pd.read_parquet("/kaggle/input/leetcode-tasks/train-00000-of-00001.parquet")

# algorithms

In [23]:
algorithms = algorithms[(algorithms["language"] == "python") | (algorithms["language"] == "javascript")]
algorithms = algorithms.drop(columns=[
    'slug',
    'difficulty',
    'qwq',
]).reset_index(drop=True)

print(algorithms["language"].value_counts())
algorithms.head(3)

language
python        397
javascript    389
Name: count, dtype: int64


,id,title,content,solution,language
0,1568,Minimum Number of Days to Disconnect Island,You are given an `m x n` binary grid `grid` wh...,"```python\ndef pseudoPalindromicPaths(root, cn...",python
1,1714,Sum Of Special Evenly-Spaced Elements In Array,You are given a **0-indexed** integer array `n...,```javascript\nfunction reorderSpaces(text) {\...,javascript
2,1326,Minimum Number of Taps to Open to Water a Garden,There is a one-dimensional garden on the x-axi...,```javascript\nfunction sumOfFlooredPairs(nums...,javascript


## Check on specific programming words

In [16]:
from typing import List

def warm_start(words: List[str], model, tokenizer):
    words = ' '.join(words)
    result = analyze_text(words, model, tokenizer)

    return result

def warm_start_gpu(words: List[str], model, tokenizer):
    words = ' '.join(words)
    result = analyze_text_gpu(words, model, tokenizer)

    return result

### DeepSeek-R1-Distill-Qwen-7B

In [ ]:
deepseek_r1_tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-7B")
deepseek_r1_model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-7B")

In [ ]:
with open("deepseek-r1-qwen-7b.txt", "w", encoding="utf-8") as f:
    results = {}
    
    for i in range(len(all_keywords)):
        keywords = all_keywords[i]
        warm_start_result = warm_start(keywords, deepseek_r1_model, deepseek_r1_tokenizer)
        results[f"language_{i}"] = warm_start_result

    json.dump(results, f, ensure_ascii=False, indent=4)

### DeepSeek-R1-Distill-Qwen-1.5B

In [5]:
device = "cuda"

deepseek_r1_tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")
deepseek_r1_model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

deepseek_r1_model.to(device)

tokenizer_config.json:   0%|          | 0.00/3.06k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qw

#### Previous way

In [17]:
with open("deepseek-r1-qwen-1500M.txt", "w", encoding="utf-8") as f:
    results = {}
    
    for i in range(len(all_keywords)):
        keywords = all_keywords[i]
        warm_start_result = warm_start_gpu(keywords, deepseek_r1_model, deepseek_r1_tokenizer)
        results[f"language_{i}"] = warm_start_result

    json.dump(results, f, ensure_ascii=False, indent=4)

In [19]:
with open("deepseek-r1-qwen-1500M-missed-tokens.txt", "w", encoding="utf-8") as f:
    results = {}
    warm_start_result = warm_start_gpu(misswords, deepseek_r1_model, deepseek_r1_tokenizer)
    results[f"misswords"] = warm_start_result

    json.dump(results, f, ensure_ascii=False, indent=4)

In [20]:
def answer_question(prompt, model, tokenizer):
    """
    Generates an answer based on the given context using a causal language model.

    Args:
        question (str): The question to be answered.
        context (str): The context in which to find the answer.
        model: The causal language model.
        tokenizer: The tokenizer for the model.

    Returns:
        str: The generated answer.
    """
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=512)

    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return answer

In [8]:
# tasks = pd.read_parquet("/kaggle/input/tokenizers-python-dataset/train-00000-of-00002.parquet")

# tasks.head(1)

In [24]:
df_solutions = pd.DataFrame()
tokens_to_give = ""

# for i in range(len(algorithms)):
for i in range(15):
    if i%5 == 0:
        print(i)

    task = algorithms["content"][i]
    task_id = algorithms["id"][i]
    task_name = algorithms["title"][i]
    language = algorithms["language"][i]
    prompt = f"Solve this programming task, you need to return only program code in {language}. Task: {task}"

    solution = answer_question(prompt, deepseek_r1_model, deepseek_r1_tokenizer)

    df_solutions.loc[i, "task_id"] = task_id
    df_solutions.loc[i, "task_name"] = task_name
    df_solutions.loc[i, "task"] = task
    df_solutions.loc[i, "language"] = language
    df_solutions.loc[i, "prompt"] = prompt
    df_solutions.loc[i, "solution"] = solution

df_solutions.to_csv("deepseek_r1_distill_qwen_1500M.csv", index=False)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


0


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


5


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In [41]:
system_prompt = "You are AI-assistant which helps user to solve programming tasks."

for i in range(1):
    task = algorithms["content"][i]
    task_id = algorithms["id"][i]
    task_name = algorithms["title"][i]
    language = algorithms["language"][i]
    prompt = f"Solve this programming task, you need to return only program code in {language}. Task: {task}"

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ]

    text = deepseek_r1_tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = deepseek_r1_tokenizer([text], return_tensors="pt").to(deepseek_r1_model.device)
    
    generated_ids = deepseek_r1_model.generate(
        **model_inputs,
        max_new_tokens=2048
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = deepseek_r1_tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    df_solutions.loc[i, "task_id"] = task_id
    df_solutions.loc[i, "task_name"] = task_name
    df_solutions.loc[i, "task"] = task
    df_solutions.loc[i, "language"] = language
    df_solutions.loc[i, "prompt"] = prompt
    df_solutions.loc[i, "solution"] = response
    print(i)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


0


In [7]:
# df_solutions

In [6]:
# df_solutions["solution"][0]

#### New way

In [11]:
def warm_start(word: str, model, tokenizer):
    """
    Process a single word and get probabilities of the next 10 tokens.
    """
    return analyze_text(word, model, tokenizer)

def analyze_text(word: str, model, tokenizer, top_k=10):
    """
    Analyze token probabilities for a given word.
    """
    results = get_token_probabilities(model, tokenizer, word, top_k)
    
    tokens_prob = {}
    if results:
        given_token = results[0]['token']
        tokens_prob[given_token] = results[0]['probabilities']
    
    return tokens_prob

def get_token_probabilities(model, tokenizer, word: str, top_k=10):
    """
    Get token probabilities for the next tokens based on a single input word.
    """
    # Tokenize input word
    input_ids = tokenizer.encode(word, return_tensors='pt').to(device)
    
    # Get model's raw predictions
    with torch.no_grad():
        outputs = model(input_ids)
        logits = outputs.logits
    
    # Take the last token's logits for next token prediction
    last_token_logits = logits[0, -1, :]
    
    # Convert logits to probabilities using softmax
    probs = F.softmax(last_token_logits, dim=-1)
    
    # Get token indices sorted by probability
    sorted_indices = torch.argsort(probs, descending=True)[:top_k]
    
    # Create dictionary of token:probability pairs
    return [{
        'token': word,
        'probabilities': {
            tokenizer.decode([idx.item()]): probs[idx].item()
            for idx in sorted_indices
        }
    }]


In [18]:
with open("deepseek-r1-qwen-1500M.txt", "w", encoding="utf-8") as f:
    results = {}

    for i in range(len(all_keywords)):
        keywords = all_keywords[i]
        results[f"language_{i}"] = {}
        for j in range(len(keywords)):
            warm_start_result = warm_start(keywords[j], deepseek_r1_model, deepseek_r1_tokenizer)
            # results[f"language_{i}"][f"{i}_{j}_{keywords[j]}"] = warm_start_result
            results[f"language_{i}"][j] = warm_start_result

    json.dump(results, f, ensure_ascii=False, indent=4)

In [21]:
with open("deepseek-r1-qwen-1500M-missed-tokens.txt", "w", encoding="utf-8") as f:
    results = {}
    results[f"misswords"] = {}
    
    for i in range(len(misswords)):
        warm_start_result = warm_start(misswords[i], deepseek_r1_model, deepseek_r1_tokenizer)
        results[f"misswords"][i] = warm_start_result

    json.dump(results, f, ensure_ascii=False, indent=4)

### DeepSeek-R1-Distill-Llama-8B 

'<think>\nOkay, I need to solve this programming problem. Let me read the problem statement carefully.\n\n'

In [ ]:
deepseek_r1_tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Llama-8B")
deepseek_r1_model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Llama-8B")

with open("deepseek-r1-llama-8b.txt", "w", encoding="utf-8") as f:
    results = {}
    
    for i in range(len(all_keywords)):
        keywords = all_keywords[i]
        warm_start_result = warm_start(keywords, deepseek_r1_model, deepseek_r1_tokenizer)
        results[f"language_{i}"] = warm_start_result

    json.dump(results, f, ensure_ascii=False, indent=4)

# Miss words in tokenizers

In [35]:
import random
from termcolor import colored

colors = ['red', 'green', 'yellow', 'blue']

def tokenize_and_colorize(tokenizer, code):
    tokens = tokenizer.tokenize(code)
    colored_code = []
    last_color = None

    for tok in tokens:
        new_color = random.choice([color for color in colors if color != last_color])
        last_color = new_color
        colored_code.append(colored(tok, new_color))

    return ' '.join(colored_code)

## Deepseek r1

In [36]:
for missword in misswords:
    colored_code = tokenize_and_colorize(deepseek_tokenizer, missword)
    print(colored_code)

non local
s ynchronized
trans ient
strict fp
fall through
complex 6 4
complex 1 2 8
float 3 2
use Effect
use Context
use Reducer
use Memo
use Callback
use Ref
use Layout Effect
use Im per ative Handle
virtual DOM
re conciliation
s usp ense
error Boundary
use Strict Mode
instance of
end declare
end for
ends witch
end while
